In [18]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import numpy as np
import pandas as pd
import time

# 連接到資料庫A ====================================================================
conn1 = sqlite3.connect('goodinfoRevenue(0_500).db')
cursor1 = conn1.cursor()
sql1 = '''select * from revenue t'''
dfData1 = pd.read_sql(sql1,conn1)
dfData1 = pd.DataFrame(dfData1)

# 連接到資料庫B ====================================================================
conn2 = sqlite3.connect('goodinfoRevenue(500_1000).db')
cursor2 = conn2.cursor()
sql2 = '''select * from revenue t'''
dfData2 = pd.read_sql(sql2,conn2)
dfData2 = pd.DataFrame(dfData2)

# 連接到資料庫C ====================================================================
conn3 = sqlite3.connect('goodinfoRevenue(500_1000).db')
cursor3 = conn3.cursor()
sql3 = '''select * from revenue t'''
dfData3 = pd.read_sql(sql3,conn3)
dfData3 = pd.DataFrame(dfData3)

# 連接到資料庫【代碼】 ==============================================================
sqlStock = '''select * from stock t'''
dfStock = pd.read_sql(sqlStock,conn1)
dfStock = pd.DataFrame(dfStock)
ids = dfStock['code']


# 【開始導入】======================================================================
# 【新增】上個月財報
thisMonth = pd.Timestamp.today() 
LastMonth = thisMonth - pd.DateOffset(months=1) # 這個月日期減上個月日期
LastMonth = LastMonth.strftime("%Y/%m") # 格式化日期

# 資料庫A
dfData1mask = dfData1['date'] == LastMonth
dfData1b = dfData1[dfData1mask]

# 資料庫B
dfData2mask = dfData2['date'] == LastMonth
dfData2b = dfData2[dfData2mask]
# 資料庫C
dfData3mask = dfData3['date'] == LastMonth
dfData3b = dfData3[dfData3mask]

# 合併資料庫A/B/C
dfconcat = pd.concat([dfData1b,dfData2b, dfData3b])
# 刪除重複的資料
dfconcat = dfconcat.drop_duplicates(subset=['code'], keep='last')


for id in ids:
    # 先查詢資料庫A資料
    dfData1Mask = dfData1['code'] == id
    dfdatabase2 = dfData1[dfData1Mask]
    dfdatabase2

    # 如果沒有上個月資料則將資料更新進資料庫
    if LastMonth not in dfdatabase2['date'].values:

        # 如果查詢的id沒有在合併資料則跳過
        if id not in dfconcat['code'].values:
            pass
        else:
            # 合併資料庫=查詢的id
            dfconcatMask = dfconcat['code'] == id
            dfconcat2 = dfconcat[dfconcatMask]

            # 將合併資料庫的id資料添加到資料庫A
            for index, row in dfconcat2.iterrows():
                try:
                    cursor1.execute(
                    """INSERT OR IGNORE INTO revenue 
                        (code,date,open,close,high,low,updownYuan,updown,revenue,mon,yoy,revenueSum,yoySum)
                        values(?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                        (row['code'],
                        row['date'],
                        row['open'],
                        row['close'],
                        row['high'],
                        row['low'],
                        row['updownYuan'],
                        row['updown'],
                        row['revenue'],
                        row['mon'],
                        row['yoy'],
                        row['revenueSum'],
                        row['yoySum'])
                        )
                    conn1.commit()
                except:
                    pass
    # 如果有的話則跳過
    else:
        pass
print('資料庫已更新成功')

資料庫已更新成功
